In [265]:
import pandas as pd 
import numpy as np
import re
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn import datasets

datapath='/home/yxr/文档/dataset/ml-100k/'

In [209]:
names="movie id | movie title | release date | video release date |IMDb URL | unknown | Action | Adventure | Animation |Children's | Comedy | Crime | Documentary | Drama | Fantasy |Film-Noir | Horror | Musical | Mystery | Romance | Sci-Fi |Thriller | War | Western |"
names=names.split('|')
names=[x.strip().replace(' ','_') for x in names]
names.remove('')
movieData=pd.read_table(datapath+'u.item',header=None,sep='|',names=names)
# movieData['age']=movieData['title'].map(lambda x: re.findall(r'[^()]+',movieData.iloc[0,1])[1])

modeValue=movieData['release_date'].mode()[0]

movieData['release_date'].fillna(modeValue,inplace=True)

movieData['release_year']=movieData['release_date'].map(lambda x : datetime.strptime(x,'%d-%b-%Y').year)

movieFeat=movieData.drop(['movie_title','release_date','video_release_date','IMDb_URL'],axis=1)
movieFeat[:5]

In [76]:
movieFeat[:4]

,movieId,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [233]:
userData=pd.read_table(datapath+'u.user',header=None,sep='|',names=['id','age','gender','occupation','zip_code'])

userData[:5]

,id,age,gender,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [230]:
bins=[7,18,26,36,61,100]
group_names=['young','youth','youngAdult','middleAge','Senior']
cats=pd.cut(userData['age'],bins,labels=group_names)
age_dummies=pd.get_dummies(cats)
occupatins=pd.get_dummies(userData['occupation'])
genders=pd.get_dummies(userData['gender'])
userFeat=userData[['id']].join([age_dummies,occupatins,genders])

In [245]:
userFeat[:5]

,id,Senior,middleAge,young,youngAdult,youth,administrator,artist,doctor,educator,...,other,programmer,retired,salesman,scientist,student,technician,writer,F,M
0,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
1,2,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,3,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
3,4,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
4,5,0,0,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0


In [234]:
uiData=pd.read_table(datapath+'u.data',header=None,names=['user_id','item_id','rating','timestamp'])

In [236]:
uiData.rating.value_counts()

4    34174
3    27145
5    21201
2    11370
1     6110
Name: rating, dtype: int64

In [238]:
uiData['rating'][uiData.rating >= 4].count()

55375

In [239]:
uiData['rating'][uiData.rating < 4].count()

44625

In [276]:
uiData['label']=uiData['rating'].map(lambda x:(x>=4))
allFeature=pd.merge(uiData[['user_id','item_id','label']],userFeat,how='left',left_on='user_id',right_on='id')
allFeature=pd.merge(allFeature,movieFeat,left_on='item_id',right_on='movie_id',how='left')
labels=allFeature['label']
Features=allFeature.drop(['id','movie_id','release_year'],axis=1)
Features_p=Features[Features.label]
Features_n=Features[~Features.label]
P_train,P_test=train_test_split(Features_p,test_size=0.2,random_state=0)
N_train,N_test=train_test_split(Features_n,test_size=0.2,random_state=0)

In [273]:
P_train.shape

(44300, 50)

In [274]:
P_test.shape

(11075, 50)

In [281]:
all_train=pd.concat([P_train,N_train])
all_train_features=all_train.drop(['label','user_id','item_id'],axis=1)
#又重新复制了一遍数据!!!! 又或者是pandas的切片,需要研究一下内部机制!!
all_train_label=all_train.label#训练集的标签数据
all_test=pd.concat([P_test,N_test])
all_test_features=all_test.drop(['label','user_id','item_id'],axis=1)
all_test_label=all_test.label

In [283]:
all_test_features.shape

(20000, 47)

In [284]:
all_train_features.shape

(80000, 47)

In [294]:
#得到特征数据和标签数据之后,开始训练模型
from sklearn.naive_bayes import BernoulliNB
bnb=BernoulliNB(binarize=None)
y_pred=bnb.fit(all_train_features,all_train_label).predict(all_test_features)
print('Number of mislabeled points out of a total %d points : %d') \
%(len(all_test_label),(all_test_label != y_pred).sum())

Number of mislabeled points out of a total 20000 points : 8421


In [295]:
trained=bnb.fit(all_train_features,all_train_label)

In [296]:
trained.class_count_

array([35700., 44300.])

In [305]:
trained.feature_count_[0].

array([3.0600e+02, 1.0571e+04, 1.6140e+03, 1.2148e+04, 1.1048e+04,
       2.4650e+03, 7.2400e+02, 1.5200e+02, 2.9310e+03, 3.0230e+03,
       8.2200e+02, 1.2810e+03, 1.4840e+03, 1.1500e+02, 3.7100e+02,
       1.8630e+03, 7.4500e+02, 2.7300e+02, 3.8440e+03, 2.6760e+03,
       6.0000e+02, 2.8400e+02, 7.0500e+02, 7.8770e+03, 1.2200e+03,
       2.2450e+03, 9.2270e+03, 2.6473e+04, 3.0000e+00, 9.6130e+03,
       5.1470e+03, 1.2490e+03, 2.9900e+03, 1.1897e+04, 2.5600e+03,
       2.3300e+02, 1.2240e+04, 6.2800e+02, 4.0500e+02, 2.2810e+03,
       1.8090e+03, 1.6580e+03, 6.4650e+03, 4.4160e+03, 7.9220e+03,
       2.5010e+03, 6.4900e+02])

In [300]:
trained.feature_log_prob_

array([[-4.75611424, -1.21699771, -3.09587175, -1.07795985, -1.17286678,
        -2.6726093 , -3.89679033, -5.45252407, -2.49952193, -2.46862625,
        -3.77000579, -3.32678535, -3.17979194, -5.7293718 , -4.56406813,
        -2.95248199, -3.86823639, -4.86983388, -2.22843311, -2.59050995,
        -4.08436705, -4.83047281, -3.92334675, -1.51113265, -3.37553651,
        -2.76605585, -1.35296437, -0.29904359, -9.09666763, -1.31198634,
        -1.93659842, -3.35206316, -2.47959893, -1.09883639, -2.6348089 ,
        -5.02764087, -1.07041574, -4.03883073, -4.47660883, -2.75015446,
        -2.98187986, -3.0689917 , -1.70864903, -2.08974598, -1.50543679,
        -2.6581163 , -4.00598963],
       [-4.63966191, -1.12047383, -3.03256318, -1.0893331 , -1.2762408 ,
        -2.52014631, -3.6803833 , -5.10407372, -2.256531  , -2.53798118,
        -3.90981336, -3.42777656, -4.04592207, -6.03534601, -4.16354383,
        -2.94045163, -4.00546143, -4.54392701, -2.256531  , -2.50898548,
        -4.18703